In [12]:
import folium
import requests
import streamlit as st 
from streamlit_folium import folium_static
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [2]:
# Getting start/dest coordinates
def get_lat_lon(streetname):
    BASE_URL = 'https://nominatim.openstreetmap.org/search?format=json'
    response = requests.get(f'{BASE_URL}&street={streetname}&city=Stuttgart')
    data = response.json()
    
    if data:
        lat = data[0].get('lat')
        lon = data[0].get('lon')
        return float(lat), float(lon)
    else:
        # Handle the case where the geocoding service does not return valid data
        return None
 
# Initializing OSM Graph
def init_osm_graph(city):
    G = ox.graph_from_place(city, network_type='bike')
    return G

G = init_osm_graph('Stuttgart')

In [3]:
# Get fastest route from OSM
def get_osm_route(start_location, dest_location):
     
    start_data = get_lat_lon(start_location)
    dest_data = get_lat_lon(dest_location)

    orig_node = ox.distance.nearest_nodes(G, start_data[1], start_data[0])
    dest_node = ox.distance.nearest_nodes(G, dest_data[1], dest_data[0])
    shortest_route = nx.shortest_path(G, orig_node, dest_node, weight="length")
    pathDistance = nx.shortest_path_length(G, orig_node, dest_node, weight='length')
    
    return shortest_route, pathDistance

In [ ]:
shortest_route, pathDistance = get_osm_route("Königstraße 4", "Tübingerstraße 40")
         
m = ox.plot_route_folium(G, shortest_route, tiles='openstreetmap')
m

In [8]:
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

In [11]:
merged_osm = pd.read_csv('OSM\merged_osm.csv')

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Zhenzhen\AppData\Local\Temp\ipykernel_16328\2845835240.py:1: SyntaxWarning: invalid escape sequence '\m'
  merged_osm = pd.read_csv('OSM\merged_osm.csv')
C:\Users\Zhenzhen\AppData\Local\Temp\ipykernel_16328\2845835240.py:1: DtypeWarning: Columns (7,8,19,20,24,25,26,27,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_osm = pd.read_csv('OSM\merged_osm.csv')


In [31]:
def db_to_graph(db_nodes, db_edges):
    # nodes = gpd.GeoDataFrame(db_nodes).set_index(["osmid"])
    nodes = db_nodes
    edges = gpd.GeoDataFrame(db_edges).set_index(["u", "v", "key", "osmid"])
    edges["geometry"] = edges["geometry"].astype(str).apply(lambda x: wkt.loads(x))
    edges = gpd.GeoDataFrame(edges).set_crs("epsg:4326")
    G = ox.graph_from_gdfs(nodes, edges)
    return G

In [33]:
merged_osm['geometry'].value_counts(dropna=False)

geometry
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [32]:
G1 = db_to_graph(nodes, merged_osm)

GEOSException: ParseException: Expected word but encountered number: 'nan'